# Second Round of Full Day RFI Flagging

**by Josh Dillon**, last updated July 28, 2023

This notebook is synthesizes information from individual [delay_filtered_average_zscore](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/delay_filtered_average_zscore.ipynb) notebooks to find low-level RFI and flag it. That notebook takes `smooth_cal`ibrated data, redundantly averages it, performs a high-pass delay filter, and then incoherently averages across baselines, creating a per-polarization z-score. This notebook then takes that whole night of z-scores and finds a new set of flags to both add to the `smooth_cal` files, which are updated in place, and to write down as new `UVFlag` waterfall-type `.h5` files.

Here's a set of links to skip to particular figures and tables:


# [• Figure 1: Waterfall of Maximum z-Score of Either Polarization Before Round 2 Flagging](#Figure-1:-Waterfall-of-Maximum-z-Score-of-Either-Polarization-Before-Round-2-Flagging)
# [• Figure 2: Histogram of z-scores](#Figure-2:-Histogram-of-z-scores)
# [• Figure 3: Waterfall of Maximum z-Score of Either Polarization After Round 2 Flagging](#Figure-3:-Waterfall-of-Maximum-z-Score-of-Either-Polarization-After-Round-2-Flagging)
# [• Figure 4: Spectra of Time-Averaged z-Scores](#Figure-4:-Spectra-of-Time-Averaged-z-Scores)
# [• Figure 5: Summary of Flags Before and After Round 2 Flagging](#Figure-5:-Summary-of-Flags-Before-and-After-Round-2-Flagging)

In [ ]:
import time
tstart = time.time()

In [ ]:
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
import h5py
import hdf5plugin  # REQUIRED to have the compression plugins available
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib
import copy
import warnings
from pyuvdata import UVFlag, UVCal
from hera_cal import utils
from hera_qm import xrfi

from IPython.display import display, HTML
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))
_ = np.seterr(all='ignore')  # get rid of red warnings
%config InlineBackend.figure_format = 'retina'

In [ ]:
# get input data file names
SUM_FILE = os.environ.get("SUM_FILE", None)
# SUM_FILE = '/lustre/aoc/projects/hera/h6c-analysis/IDR2/2459861/zen.2459861.25297.sum.uvh5'
SUM_SUFFIX = os.environ.get("SUM_SUFFIX", 'sum.uvh5')

# get input and output suffixes
SMOOTH_CAL_SUFFIX = os.environ.get("CAL_SUFFIX", 'sum.smooth.calfits')
ZSCORE_SUFFIX =  os.environ.get("ZSCORE_SUFFIX", 'sum.red_avg_zscore.h5')
FLAG_WATERFALL2_SUFFIX = os.environ.get("FLAG_WATERFALL2_SUFFIX", 'sum.flag_waterfall_round_2.h5')

# build globs
sum_glob = '.'.join(SUM_FILE.split('.')[:-3]) + '.*.' + SUM_SUFFIX
cal_files_glob = sum_glob.replace(SUM_SUFFIX, SMOOTH_CAL_SUFFIX)
zscore_glob = sum_glob.replace(SUM_SUFFIX, ZSCORE_SUFFIX)

# get flagging parameters
Z_THRESH = float(os.environ.get("Z_THRESH", 5))
WS_Z_THRESH = float(os.environ.get("WS_Z_THRESH", 4))
AVG_Z_THRESH = float(os.environ.get("AVG_Z_THRESH", 1))
MAX_FREQ_FLAG_FRAC = float(os.environ.get("MAX_FREQ_FLAG_FRAC", .25))
MAX_TIME_FLAG_FRAC = float(os.environ.get("MAX_TIME_FLAG_FRAC", .1))

for setting in ['Z_THRESH', 'WS_Z_THRESH', 'AVG_Z_THRESH', 'MAX_FREQ_FLAG_FRAC', 'MAX_TIME_FLAG_FRAC']:
    print(f'{setting} = {eval(setting)}')

# Load z-scores

In [ ]:
# load z-scores
zscore_files = sorted(glob.glob(zscore_glob))
print(f'Found {len(zscore_files)} *.{ZSCORE_SUFFIX} files starting with {zscore_files[0]}.')
uvf = UVFlag(zscore_files, use_future_array_shapes=True)

In [ ]:
# get calibration solution files
cal_files = sorted(glob.glob(cal_files_glob))
print(f'Found {len(cal_files)} *.{SMOOTH_CAL_SUFFIX} files starting with {cal_files[0]}.')

In [ ]:
assert len(zscore_files) == len(cal_files)

In [ ]:
# extract z-scores and correct by a single number per polarization to account for biases created by filtering
zscore = {pol: uvf.metric_array[:, :, np.argwhere(uvf.polarization_array == utils.polstr2num(pol, x_orientation=uvf.x_orientation))[0][0]] for pol in ['ee', 'nn']}
zscore = {pol: zscore[pol] - np.nanmedian(zscore[pol]) for pol in zscore}

In [ ]:
freqs = uvf.freq_array
times = uvf.time_array

In [ ]:
extent = [freqs[0] / 1e6, freqs[-1] / 1e6, times[-1] - int(times[0]), times[0] - int(times[0])]

In [ ]:
def plot_max_z_score(zscore, flags=None):
    if flags is None:
        flags = np.any(~np.isfinite(list(zscore.values())), axis=0)
    plt.figure(figsize=(14,10), dpi=100)
    plt.imshow(np.where(flags, np.nan, np.nanmax([zscore['ee'], zscore['nn']], axis=0)), aspect='auto', 
               cmap='coolwarm', interpolation='none', vmin=-10, vmax=10, extent=extent)
    plt.colorbar(location='top', label='Max z-score of either polarization', extend='both', aspect=40, pad=.02)
    plt.xlabel('Frequency (MHz)')
    plt.ylabel(f'JD - {int(times[0])}')
    plt.tight_layout()

# *Figure 1: Waterfall of Maximum z-Score of Either Polarization Before Round 2 Flagging*

Shows the worse of the two results from [delay_filtered_average_zscore](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/delay_filtered_average_zscore.ipynb) from either polarization. Dips near flagged channels are expected, due to overfitting of noise. Positive-going excursions are problematic and likely evidence of RFI.

In [ ]:
plot_max_z_score(zscore)

In [ ]:
def plot_histogram():
    plt.figure(figsize=(14,4), dpi=100)
    bins = np.arange(-50, 100, .1)
    hist_ee = plt.hist(np.ravel(zscore['ee']), bins=bins, density=True, label='ee-polarized z-scores', alpha=.5)
    hist_nn = plt.hist(np.ravel(zscore['nn']), bins=bins, density=True, label='nn-polarized z-scores', alpha=.5)
    plt.plot(bins, (2*np.pi)**-.5 * np.exp(-bins**2 / 2), 'k:', label='Gaussian approximate\nnoise-only distribution')
    plt.axvline(WS_Z_THRESH, c='r', ls='--', label='Watershed z-score')
    plt.axvline(Z_THRESH, c='r', ls='-', label='Threshold z-score')
    plt.yscale('log')
    all_densities = np.concatenate([hist_ee[0][hist_ee[0] > 0], hist_nn[0][hist_nn[0] > 0]]) 
    plt.ylim(np.min(all_densities) / 2, np.max(all_densities) * 2)
    plt.xlim([-50, 100])
    plt.legend()
    plt.xlabel('z-score')
    plt.ylabel('Density')
    plt.tight_layout()

# *Figure 2: Histogram of z-scores*

Shows a comparison of the histogram of z-scores in this file (one per polarization) to a Gaussian approximation of what one might expect from thermal noise. Without filtering, the actual distribution is a weighted sum of Rayleigh distributions. Filtering further complicates this. To make the z-scores more reliable, a single per-polarization median is subtracted from each waterfall, which allows us to flag low-level outliers with more confidence. Any points beyond the solid red line are flagged. Any points neighboring a flag beyond the dashed red line are also flagged. Finally, flagging is performed for low-level outliers in whole times or channels.

In [ ]:
plot_histogram()

## Perform flagging

In [ ]:
def iteratively_flag_on_averaged_zscore(flags, zscore, avg_z_thresh=1.5, verbose=True):
    '''Flag whole integrations or channels based on average z-score. This is done
    iteratively to prevent bad times affecting channel averages or vice versa.'''
    flagged_chan_count = 0
    flagged_int_count = 0
    while True:
        zspec = np.nanmean(np.where(flags, np.nan, zscore), axis=0)
        ztseries = np.nanmean(np.where(flags, np.nan, zscore), axis=1)

        if (np.nanmax(zspec) < avg_z_thresh) and (np.nanmax(ztseries) < avg_z_thresh):
            break

        if np.nanmax(zspec) >= np.nanmax(ztseries):
            flagged_chan_count += np.sum((zspec >= max(ztseries)) & (zspec >= avg_z_thresh))
            flags[:, (zspec >= max(ztseries)) & (zspec >= avg_z_thresh)] = True
        else:
            flagged_int_count += np.sum((ztseries >= max(zspec)) & (ztseries >= avg_z_thresh))
            flags[(ztseries >= max(zspec)) & (ztseries >= avg_z_thresh), :] = True

    if verbose:
        print(f'\tFlagging an additional {flagged_int_count} integrations and {flagged_chan_count} channels.')

def impose_max_chan_flag_frac(flags, max_flag_frac=.25, verbose=True):
    '''Flag channels already flagged more than max_flag_frac (excluding completely flagged times).'''
    unflagged_times = ~np.all(flags, axis=1)
    frequently_flagged_chans =  np.mean(flags[unflagged_times, :], axis=0) >= max_flag_frac
    if verbose:
        print(f'\tFlagging {np.sum(frequently_flagged_chans) - np.sum(np.all(flags, axis=0))} channels previously flagged {max_flag_frac:.2%} or more.')        
    flags[:, frequently_flagged_chans] = True 
        
def impose_max_time_flag_frac(flags, max_flag_frac=.25, verbose=True):
    '''Flag times already flagged more than max_flag_frac (excluding completely flagged channels).'''
    unflagged_chans = ~np.all(flags, axis=0)
    frequently_flagged_times =  np.mean(flags[:, unflagged_chans], axis=1) >= max_flag_frac
    if verbose:
        print(f'\tFlagging {np.sum(frequently_flagged_times) - np.sum(np.all(flags, axis=1))} times previously flagged {max_flag_frac:.2%} or more.')
    flags[frequently_flagged_times, :] = True             

In [ ]:
flags = np.any(~np.isfinite(list(zscore.values())), axis=0)
print(f'{np.mean(flags):.3%} of waterfall flagged to start.')

# flag largest outliers
for pol in ['ee', 'nn']:
    flags |= (zscore[pol] > Z_THRESH) 
print(f'{np.mean(flags):.3%} of waterfall flagged after flagging z > {Z_THRESH} outliers.')
    
# watershed flagging
while True:
    nflags = np.sum(flags)
    for pol in ['ee', 'nn']:
        flags |= xrfi._ws_flag_waterfall(zscore[pol], flags, WS_Z_THRESH)
    if np.sum(flags) == nflags:
        break
print(f'{np.mean(flags):.3%} of waterfall flagged after watershed flagging on z > {WS_Z_THRESH} neightbors of prior flags.')
        
# flag whole integrations or channels
while True:
    nflags = np.sum(flags)
    for pol in ['ee', 'nn']:    
        iteratively_flag_on_averaged_zscore(flags, zscore[pol], avg_z_thresh=AVG_Z_THRESH, verbose=True)
        impose_max_chan_flag_frac(flags, max_flag_frac=MAX_FREQ_FLAG_FRAC, verbose=True)
        impose_max_time_flag_frac(flags, max_flag_frac=MAX_TIME_FLAG_FRAC, verbose=True)
    if np.sum(flags) == nflags:
        break  
print(f'{np.mean(flags):.3%} of waterfall flagged after flagging whole times and channels with average z > {AVG_Z_THRESH}.')

## Show results of flagging

# *Figure 3: Waterfall of Maximum z-Score of Either Polarization After Round 2 Flagging*

The same as Figure 1, but after the flagging performed in this notebook.

In [ ]:
plot_max_z_score(zscore, flags=flags)

In [ ]:
def zscore_spectra():
    fig, axes = plt.subplots(2, 1, figsize=(14,6), dpi=100, sharex=True, sharey=True, gridspec_kw={'hspace': 0})
    for ax, pol in zip(axes, ['ee', 'nn']):

        ax.plot(freqs / 1e6, np.nanmean(zscore[pol], axis=0),'r', label=f'{pol}-Polarization Before Round 2 Flagging', lw=.5)
        ax.plot(freqs / 1e6, np.nanmean(np.where(flags, np.nan, zscore[pol]), axis=0), label=f'{pol}-Polarization After Round 2 Flagging')
        ax.legend(loc='lower right')
        ax.set_ylabel('Time-Averged Z-Score\n(Excluding Flags)')
        ax.set_ylim(-11, 11)
    axes[1].set_xlabel('Frequency (MHz)')
    plt.tight_layout()

# *Figure 4: Spectra of Time-Averaged z-Scores*

The average along the time axis of Figures 1 and 3 (though now separated per-polarization). This plot is useful for showing channels with repeated low-level RFI.

In [ ]:
zscore_spectra()

In [ ]:
def summarize_flagging():
    plt.figure(figsize=(14,10), dpi=100)
    cmap = matplotlib.colors.ListedColormap(((0, 0, 0),) + matplotlib.cm.get_cmap("Set2").colors[0:2])
    plt.imshow(np.where(np.any(~np.isfinite(list(zscore.values())), axis=0), 1, np.where(flags, 2, 0)), 
               aspect='auto', cmap=cmap, interpolation='none', extent=extent)
    plt.clim([-.5, 2.5])
    cbar = plt.colorbar(location='top', aspect=40, pad=.02)
    cbar.set_ticks([0, 1, 2])
    cbar.set_ticklabels(['Unflagged', 'Previously Flagged', 'Flagged Here Using Delayed Filtered z-Scores'])
    plt.xlabel('Frequency (MHz)')
    plt.ylabel(f'JD - {int(times[0])}')
    plt.tight_layout()

# *Figure 5: Summary of Flags Before and After Round 2 Flagging*

This plot shows which times and frequencies were flagged before and after this notebook. It is directly comparable to Figure 5 of the first round [full_day_rfi](https://github.com/HERA-Team/hera_notebook_templates/blob/master/notebooks/full_day_rfi.ipynb) notebook.

In [ ]:
summarize_flagging()

## Save results

In [ ]:
add_to_history = 'by full_day_rfi_round_2 notebook with the following environment:\n' + '=' * 65 + '\n' + os.popen('conda env export').read() + '=' * 65

In [ ]:
tind = 0
for cal_file in cal_files:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")    
        
        # update cal_file
        uvc = UVCal()
        uvc.read(cal_file, use_future_array_shapes=True)
        uvc.flag_array |= (flags[tind:tind + len(uvc.time_array), :].T)[None, :, :, None]
        uvc.history += 'Modified ' + add_to_history
        uvc.write_calfits(cal_file, clobber=True)

        # Create new flag object
        uvf = UVFlag(uvc, waterfall=True, mode='flag')
        uvf.flag_array |= flags[tind:tind + len(uvc.time_array), :, None]
        uvf.history += 'Produced ' + add_to_history
        uvf.write(cal_file.replace(SMOOTH_CAL_SUFFIX, FLAG_WATERFALL2_SUFFIX), clobber=True)
        
        # increment time index
        tind += len(uvc.time_array)

print(f'Saved {len(cal_files)} *.{FLAG_WATERFALL2_SUFFIX} files starting with {cal_files[0].replace(SMOOTH_CAL_SUFFIX, FLAG_WATERFALL2_SUFFIX)}.')

## Metadata

In [ ]:
for repo in ['hera_cal', 'hera_qm', 'hera_filters', 'hera_notebook_templates', 'pyuvdata']:
    exec(f'from {repo} import __version__')
    print(f'{repo}: {__version__}')

In [ ]:
print(f'Finished execution in {(time.time() - tstart) / 60:.2f} minutes.')